# Retreive All Grain Data

In [6]:
import os, sys, json, glob
import shutil
from itertools import product, compress, chain
from functools import reduce
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import requests
import numpy as np
import pandas as pd
pd.set_option('mode.chained_assignment', None)
import geopandas as gpd
import plotly
import plotly.graph_objects as go
import plotly.express as px

In [2]:
# R011: Cereals
# R0111: Wheat
# R0112: Maize
# R0113: Rice
# R0114: Sorghum
# R0115: Barley
# R0116: Rye
# R0117: Oats
# R0118: Millet
# R0119: Other cereals

In [5]:
for cpv_code in ['R0111','R0112','R0113','R0114','R0115','R0116','R0117','R0118','R0119']:
    # Retrieve all grain data using API
    host = 'https://fdw.fews.net'
    auth = tuple(json.loads(open('token.json', "r").read()))
    parameters = {
        'format': 'json',
        'product': '%s' % cpv_code,
        'survey_type': 'crop:best',
        'fields': 'simple'
    }
    endpoint = '/api/cropproductionfacts/'
    response = requests.get(host + endpoint, auth=auth, params=parameters, proxies={})
    response.raise_for_status()
    df = pd.DataFrame.from_records(response.json())
    
    # Save as a csv file
    fn_out = './data/crop/raw/cropproductionfacts_%s.csv' % cpv_code
    df.to_csv(fn_out)
    print('%s is saved.' % fn_out)

./data/crop/raw/cropproductionfacts_R0111.csv is saved.
./data/crop/raw/cropproductionfacts_R0112.csv is saved.
./data/crop/raw/cropproductionfacts_R0113.csv is saved.
./data/crop/raw/cropproductionfacts_R0114.csv is saved.
./data/crop/raw/cropproductionfacts_R0115.csv is saved.
./data/crop/raw/cropproductionfacts_R0116.csv is saved.
./data/crop/raw/cropproductionfacts_R0117.csv is saved.
./data/crop/raw/cropproductionfacts_R0118.csv is saved.
./data/crop/raw/cropproductionfacts_R0119.csv is saved.


In [14]:
container = []
for fn in sorted(glob.glob('./data/crop/raw/*')):
    print(fn, 'is loaded')
    df = pd.read_csv(fn,low_memory=False, index_col=0)
    # print(df.head())
    container.append(df)
df = pd.concat(container, axis=0).reset_index(drop=True)

./data/crop/raw/cropproductionfacts_R0111.csv is loaded
./data/crop/raw/cropproductionfacts_R0112.csv is loaded
./data/crop/raw/cropproductionfacts_R0113.csv is loaded
./data/crop/raw/cropproductionfacts_R0114.csv is loaded
./data/crop/raw/cropproductionfacts_R0115.csv is loaded
./data/crop/raw/cropproductionfacts_R0116.csv is loaded
./data/crop/raw/cropproductionfacts_R0117.csv is loaded
./data/crop/raw/cropproductionfacts_R0118.csv is loaded
./data/crop/raw/cropproductionfacts_R0119.csv is loaded


In [15]:
df.shape

(2991672, 37)

In [16]:
df

,country,source_organization,source_document,fnid,admin_0,admin_1,admin_2,admin_3,admin_4,population_group,...,collection_schedule,collection_date,season_date,season_year,status,collection_status,publication_name,document_type,id,dataseries_name
0,Afghanistan,"Ministry of Agriculture, Irrigation and Livest...",Afghanistan Agricultural Annual Report,AF2005A101,Afghanistan,Kabul,NaN,NaN,NaN,NaN,...,Ad Hoc,2018-08-31,2018-07-01,Summer harvest 2018,Not Collected,Published,"Agricultural Annual Report 2018-19, January 2019",crop_final,17344216,"Afghanistan Agricultural Annual Report: Kabul,..."
1,Afghanistan,"Ministry of Agriculture, Irrigation and Livest...",Afghanistan Agricultural Annual Report,AF2005A101,Afghanistan,Kabul,NaN,NaN,NaN,NaN,...,Ad Hoc,2018-07-31,2018-05-01,Spring harvest 2018,Not Collected,Published,"Agricultural Annual Report 2018-19, January 2019",crop_final,17344080,"Afghanistan Agricultural Annual Report: Kabul,..."
2,Afghanistan,"Ministry of Agriculture, Irrigation and Livest...",Afghanistan Agricultural Annual Report,AF2005A101,Afghanistan,Kabul,NaN,NaN,NaN,NaN,...,Ad Hoc,2018-08-31,2018-07-01,Summer harvest 2018,Collected,Published,"Agricultural Annual Report 2018-19, January 2019",crop_final,17344217,"Afghanistan Agricultural Annual Report: Kabul,..."
3,Afghanistan,"Ministry of Agriculture, Irrigation and Livest...",Afghanistan Agricultural Annual Report,AF2005A101,Afghanistan,Kabul,NaN,NaN,NaN,NaN,...,Ad Hoc,2018-07-31,2018-05-01,Spring harvest 2018,Collected,Published,"Agricultural Annual Report 2018-19, January 2019",crop_final,17344081,"Afghanistan Agricultural Annual Report: Kabul,..."
4,Afghanistan,"Ministry of Agriculture, Irrigation and Livest...",Afghanistan Agricultural Annual Report,AF2005A101,Afghanistan,Kabul,NaN,NaN,NaN,NaN,...,Ad Hoc,2018-08-31,2018-07-01,Summer harvest 2018,Not Collected,Published,"Agricultural Annual Report 2018-19, January 2019",crop_final,17344218,"Afghanistan Agricultural Annual Report: Kabul,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2991667,United Kingdom,"Department for Environment, Food & Rural Affai...","Eurostat, United Kingdom",GB2016A112,United Kingdom,North East,NaN,NaN,NaN,NaN,...,Ad Hoc,2016-08-31,2016-03-01,Spring 2016,Collected,Submitted,"Agriculture in the English Regions, United Kin...",crop_final,18424013,"Eurostat, United Kingdom: North East, United K..."
2991668,United Kingdom,"Department for Environment, Food & Rural Affai...","Eurostat, United Kingdom",GB2016A112,United Kingdom,North East,NaN,NaN,NaN,NaN,...,Ad Hoc,2019-08-31,2019-03-01,Spring 2019,Not Available,Submitted,"Agriculture in the English Regions, United Kin...",crop_final,18424830,"Eurostat, United Kingdom: North East, United K..."
2991669,United Kingdom,"Department for Environment, Food & Rural Affai...","Eurostat, United Kingdom",GB2016A112,United Kingdom,North East,NaN,NaN,NaN,NaN,...,Ad Hoc,2018-08-31,2018-03-01,Spring 2018,Collected,Submitted,"Agriculture in the English Regions, United Kin...",crop_final,18424774,"Eurostat, United Kingdom: North East, United K..."
2991670,United Kingdom,"Department for Environment, Food & Rural Affai...","Eurostat, United Kingdom",GB2016A112,United Kingdom,North East,NaN,NaN,NaN,NaN,...,Ad Hoc,2017-08-31,2017-03-01,Spring 2017,Collected,Submitted,"Agriculture in the English Regions, United Kin...",crop_final,18424398,"Eurostat, United Kingdom: North East, United K..."
